# Drop all columns from template except those needed

### Description

- Use when only keeping a portion of the dataset contained within a template
- Can also use for large dataset cleaning
- Filter based on header name, header string contents, location in sheet, column value, no header contents, or a combination of these
- Useful when template contains more variables than needed, columns with no header label & contents aren't needed, columns with non-data entries
- Can incorporate into cleaning function when iterating through files in a directory

## Import libraries

In [1]:
import pandas as pd

## Import data

In [2]:
dfa1 = pd.read_csv('columns_template_raw.csv')

In [3]:
dfa1

,sector,id,Unnamed: 2,loc_1,loc_2,number,Unnamed: 6,letter_1,color,count_1,count_2
0,hq,P1.1,NaN,north,up,1,NaN,a,red,24,156
1,hq,P1.2,blank,south,down,2,NaN,b,orange,32,213
2,labor,P1.3,NaN,east,left,3,blank,c,yellow,52,783
3,labor,P1.4,NaN,west,right,4,NaN,d,green,30,298


In [4]:
dfa1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sector      4 non-null      object
 1   id          4 non-null      object
 2   Unnamed: 2  1 non-null      object
 3   loc_1       4 non-null      object
 4   loc_2       4 non-null      object
 5   number      4 non-null      int64 
 6   Unnamed: 6  1 non-null      object
 7   letter_1    4 non-null      object
 8   color       4 non-null      object
 9   count_1     4 non-null      int64 
 10  count_2     4 non-null      int64 
dtypes: int64(3), object(8)
memory usage: 484.0+ bytes


## Keep only columns needed in cleaned template

In [5]:
# Filter to keep only columns with certain headers names
dfa2 = dfa1[['sector', 'id', 'loc_1', 'loc_2', 'color']]

In [6]:
dfa2

,sector,id,loc_1,loc_2,color
0,hq,P1.1,north,up,red
1,hq,P1.2,south,down,orange
2,labor,P1.3,east,left,yellow
3,labor,P1.4,west,right,green


In [7]:
# Filter to keep columns based on header string contents
dfa3 = dfa1.filter(like = 'loc') # Can put any non-regex string contents here

# OR

# Useful to specify multiple conditions
header_string_contents = dfa1.columns.str.contains('loc')
dfa3 = dfa1.loc[:, header_string_contents] # Can add case = False to ignore caps

In [8]:
dfa3

,loc_1,loc_2
0,north,up
1,south,down
2,east,left
3,west,right


In [9]:
# Filter to keep columns based on header string contents WITH REGEX for more complexity
dfa4 = dfa1.filter(regex = '.*_\d+') # This regex keeps any header that is a string followed by underscore & number

In [10]:
dfa4

,loc_1,loc_2,letter_1,count_1,count_2
0,north,up,a,24,156
1,south,down,b,32,213
2,east,left,c,52,783
3,west,right,d,30,298


In [11]:
# Filter columns by location in dataframe: exact column position
dfa5 = dfa1.iloc[:, [0, 1, 8]]

In [12]:
dfa5

,sector,id,color
0,hq,P1.1,red
1,hq,P1.2,orange
2,labor,P1.3,yellow
3,labor,P1.4,green


In [13]:
# Filter columns by location in dataframe: relational column position
reference_header = dfa1.columns.get_loc('loc_1')
dfa6 = dfa1.iloc[:, reference_header : reference_header + 3] # If you want all to right, just do reference_header:

In [14]:
dfa6

,loc_1,loc_2,number
0,north,up,1
1,south,down,2
2,east,left,3
3,west,right,4


In [15]:
# Filter columns by value type
dfa7 = dfa1.select_dtypes(include = 'number')

In [16]:
dfa7

,number,count_1,count_2
0,1,24,156
1,2,32,213
2,3,52,783
3,4,30,298


In [17]:
# Filter columns by condition on value
dfa7 = dfa1.select_dtypes(include = 'number') # Need to filter out non-numeric columns first
dfa8 = dfa7.loc[:, (dfa7 < 100).any()] # Keep only columns with any values below 100- change column call to whatever filter is needed

In [18]:
dfa8

,number,count_1
0,1,24
1,2,32
2,3,52
3,4,30


In [19]:
# Remove columns with no header name
dfa9 = dfa1.loc[:, ~dfa1.columns.str.contains('Unnamed')]

In [20]:
dfa9

,sector,id,loc_1,loc_2,number,letter_1,color,count_1,count_2
0,hq,P1.1,north,up,1,a,red,24,156
1,hq,P1.2,south,down,2,b,orange,32,213
2,labor,P1.3,east,left,3,c,yellow,52,783
3,labor,P1.4,west,right,4,d,green,30,298


In [21]:
# Filter based on a combination of the above without merging
condition_1 = dfa1.filter(like = 'loc').columns.tolist() # Create list with columns matching this condition
condition_2 = dfa1.select_dtypes(include = 'number').columns.tolist() # Create list with columns matching this condition
conditions = list(set(condition_1 + condition_2)) # Combine conditional column lists
dfa10 = dfa1[conditions] # Create new df with only columns matching above conditions

In [22]:
dfa10

,number,loc_2,loc_1,count_1,count_2
0,1,up,north,24,156
1,2,down,south,32,213
2,3,left,east,52,783
3,4,right,west,30,298


In [23]:
# Filter based on a combination of the above with merging
# Useful to be able to parse numerical columns separately & add certain categorical columns back after

# Create smaller df with numerical data & keep only columns that match certain conditions
dfb1 = dfa1.select_dtypes(include = 'number')
condition_1 = (dfb1 < 100).any()
condition_2 = (dfb1 % 2 == 0).all()
conditions = condition_1 & condition_2
dfb2 = dfb1.loc[:, conditions]

# Create smaller df with desired categorical columns
dfa11 = dfa1[['sector', 'id']]

# Merge the dfs back together- join merges by index
dfa12 = dfa11.join(dfb2)

In [24]:
dfa12

,sector,id,count_1
0,hq,P1.1,24
1,hq,P1.2,32
2,labor,P1.3,52
3,labor,P1.4,30


## Export data

In [25]:
dfa2.to_csv('cleaned_columns_header_names.csv', encoding = 'utf-8', index = False, header = True)

dfa3.to_csv('cleaned_columns_header_string_contents.csv', encoding = 'utf-8', index = False, header = True)

dfa4.to_csv('cleaned_columns_header_regex.csv', encoding = 'utf-8', index = False, header = True)

dfa5.to_csv('cleaned_columns_exact_position.csv', encoding = 'utf-8', index = False, header = True)

dfa6.to_csv('cleaned_columns_relational_position.csv', encoding = 'utf-8', index = False, header = True)

dfa7.to_csv('cleaned_columns_value_type.csv', encoding = 'utf-8', index = False, header = True)

dfa8.to_csv('cleaned_columns_value_condition.csv', encoding = 'utf-8', index = False, header = True)

dfa9.to_csv('cleaned_columns_no_blank_headers.csv', encoding = 'utf-8', index = False, header = True)

dfa10.to_csv('cleaned_columns_multiple_conditions.csv', encoding = 'utf-8', index = False, header = True)

dfa12.to_csv('cleaned_columns_categorical_and_numerical_conditions.csv', encoding = 'utf-8', index = False, header = True)